
## FINANCIAL DATA
MODULE 5 | LESSON 1


---



# **DOWNLOADING, CLEANING, AND TRANSFORMING DATA**

|  |  |
|:---|:---|
|**Reading Time** |  30 minutes |
|**Prior Knowledge** | Basic Python, Probability of default (PD)  |
|**Keywords** |df.dropna(), inplace, df.replace(), .astype(), pd.to_datetime(), df['X'].dt.strftime, masking / filtering, df.concat(), <br> .append, df.values.tolist(), datetime.strptime, datetime.datetime|

---



*In this module, we perform the several steps required to calculate the market-implied probability of default (PD), which we introduced in Module 5.*
*In this lesson, we will indicate the company (bond issuer) and determine which of the company's bonds to use after cleaning, transforming, and filtering the data.*

*In the lessons that follow in this module, we will estimate the expected cashflows and the risk-adjusted discount rate (lesson 2). Finally we estimate the market-implied probability of default (lesson 3) and compare it to the PD that Standard and Poor's (S&P) associates with the rating (lesson 4).*<span style='color: transparent; font-size:1%'>All rights reserved WQU WorldQuant University QQQQ</span>

## 1. Re-introduction to Probability of Default (PD)

As we discussed in the Financial Markets course, the probability of default is the probability that a bond issuer will not meet its contractual obligations on schedule. Although the most common event of default is nonpayment leading to bankruptcy proceedings, the bond prospectus might identify other events of default, such as the failure to meet a different obligation or the violation of a financial covenant.

In the following example, we will determine the probability of default given corporate bond prices. The default probabilities that are reached in this exercise are called market-implied default probabilities. Historically, practitioners have focused on the one-year probability of default calculation. Over shorter horizons of one or two years, firms are exposed to the business cycle effect, while over longer horizons, the business cycle effect tends to have a lesser impact and the company's capital structure becomes more important. This effect has made long-run risk levels less cyclical and more stable. Intuitively, default risk over a longer time period is less sensitive to the instantaneous default rates in the economy (Beygi 3). For this reason, we will focus on corporate bonds with one or two years until maturity to calculate the market-implied default probabilities.

We will verify the accuracy of the market-implied default probabilities with the Standard & Poor's "Average One-Year Transition Rates For Global Corporates" table, which uses historical data from 1981-2019. This transition matrix shows the observed historical probabilities of a particular rating transitioning to another rating, including default, over the course of one year.

In order to calculate the market-implied default probabilities, we must first acquire the company's current bond prices. Using a short Playwright script that emulates a user's keystrokes and clicks in a browser as a means of navigating to Trade Reporting and Compliance Engine (TRACE) bond data provided by the Financial Industry Regulatory Authority (FINRA), we can access the data needed to calculate the market-implied default probabilities.

## 2. Playwright

Playwright is a tool used to automate browser activities typically performed by a user, such as loading a web page and filling out a form.

The choice of Playwright over Selenium lies in its ease of use for our purposes. It offers several advantages, especially when dealing with complex web applications that utilize multiple nested shadow DOMs, similar to the FINRA web page we will be scraping for bond data. One of the primary strengths of Playwright is its locator functionality, which is designed to handle these shadow DOMs with less hassle compared to Selenium's traditional selector methods.

Furthermore, Playwright's auto-waiting feature is inherently built into its actions. This means that Playwright automatically waits for elements to be ready before performing actions on them, which is essential for dynamic content loading and asynchronous UIs. In contrast, Selenium often requires explicit waits or sleep statements, which can lead to either increased execution time or flakiness due to timing issues.

In [ ]:
# Python libraries to install
from datetime import date
from datetime import datetime as dt

import numpy as np
import pandas as pd

# The use of the async_api is mandatory since we will be running the Playwright code from inside a Jupyter Notebook cell.
from playwright.async_api import async_playwright

In [ ]:
# Input of issuer name (corporations that are subsidiaries of HESS CORP and contain the name 'hess' will be included as well)
issuer_name = "hess"

# Maturity dates
date_start = "2028-01-01"
date_end = "2050-01-01"

In [ ]:
scrape_new_data = True


async def scrape_bond_data(issuer_name, date_start, date_end):
    async with async_playwright() as pw:
        browser = await pw.chromium.launch(headless=True)
        context = await browser.new_context(viewport={"width": 1920, "height": 1080})
        page = await context.new_page()

        # Navigate to the page
        await page.goto("https://www.finra.org/finra-data/fixed-income/corp-and-agency")

        # Fill in the search criteria
        await page.locator('input[placeholder="Issuer Name"]').fill(issuer_name)
        await page.locator("input#form-name").nth(0).fill(date_start)
        await page.locator("input#form-name").nth(1).fill(date_end)

        # Submit the search form
        await page.locator('text="Show Results"').click()
        await page.wait_for_timeout(5000)  # Adjust the timeout as necessary

        # Extend the container for the data grid to ensure all data is visible
        await page.locator("ag-grid-angular").evaluate(
            """
            element => {
                element.style.height = "1500px";
                element.style.width = "4500px";
            }
        """
        )
        await page.wait_for_timeout(2000)  # Adjust the timeout as necessary

        # Extract the data
        column_ids = [
            "issueSymbolIdentifier",
            "issuerName",
            "isCallable",
            "productSubTypeCode",
            "couponRate",
            "maturityDate",
            "industryGroup",
            "moodysRating",
            "standardAndPoorsRating",
            "lastSalePrice",
            "lastSaleYield",
        ]

        data = {}
        for column_id in column_ids:
            column_data = await page.locator(
                f"div[col-id='{column_id}']"
            ).all_text_contents()
            data[column_id] = column_data[1:]  # Skip the header

        # Close the browser
        await browser.close()

        # Convert to a pandas DataFrame
        return pd.DataFrame.from_dict(data)


# Run the function
if scrape_new_data:
    bond_prices_df = await scrape_bond_data(  # noQA F704
        issuer_name, date_start, date_end
    )
else:
    bond_prices_df = pd.read_csv("bond-prices.csv")

In [ ]:
bond_prices_df

## 3. Cleaning, Transforming, and Filtering

We will now filter the corporate bond DataFrame to highlight the fact that the data is not always given in a usable form.

In [ ]:
# We observe that all columns are identified as strings or as mixed data types(as indicated from Dtype: object)
bond_prices_df.info()

In [ ]:
# We expect that the columns: "couponRate", "lastSalePrice" and "lastSaleYield" are floats.
bond_prices_df["couponRate"] = pd.to_numeric(bond_prices_df["couponRate"])
bond_prices_df["lastSalePrice"] = pd.to_numeric(bond_prices_df["lastSalePrice"])
bond_prices_df["lastSaleYield"] = pd.to_numeric(bond_prices_df["lastSaleYield"])

# The column "maturityDate" can also be converted to "Timestamp" or "Datetime" object.
bond_prices_df["maturityDate"] = pd.to_datetime(
    bond_prices_df["maturityDate"], infer_datetime_format=True
)

In [ ]:
# The datatypes of the columns after cleaning
bond_prices_df.info()

In [ ]:
# This cell will be used in case the scrapper does not work due to changes in the FINRA bond data web page layout.


def bond_dataframe_filter(df):
    # Drop bonds with missing yields and missing credit ratings
    df["Yield"].replace("", np.nan, inplace=True)
    df["Moody's®"].replace({"WR": np.nan, "": np.nan}, inplace=True)
    df["S&P"].replace({"NR": np.nan, "": np.nan}, inplace=True)
    df = df.dropna(subset=["Yield"])
    df = df.dropna(subset=["Moody's®"])
    df = df.dropna(subset=["S&P"])

    # Create Maturity Years column that aligns with Semi-Annual Payments from corporate bonds
    df["Yield"] = df["Yield"].astype(float)
    df["Coupon"] = df["Coupon"].astype(float)
    df["Price"] = df["Price"].astype(float)
    now = dt.strptime(date.today().strftime("%m/%d/%Y"), "%m/%d/%Y")
    df["Maturity"] = pd.to_datetime(df["Maturity"]).dt.strftime("%m/%d/%Y")
    daystillmaturity = []
    yearstillmaturity = []
    for maturity in df["Maturity"]:
        daystillmaturity.append((dt.strptime(maturity, "%m/%d/%Y") - now).days)
        yearstillmaturity.append((dt.strptime(maturity, "%m/%d/%Y") - now).days / 360)
    df = df.reset_index(drop=True)
    df["Maturity"] = pd.Series(daystillmaturity)
    df["Maturity Years"] = (
        round(pd.Series(yearstillmaturity) / 0.5) * 0.5
    )  # Better for Semi-Annual Payments

    # Target bonds with short-term maturities
    df["Maturity"] = df["Maturity"].astype(float)
    years_mask = (df["Maturity Years"] > 0) & (df["Maturity Years"] <= 5)
    df = df.loc[years_mask]
    return df


# The next line of code will run if scrape_new_data = False
if not scrape_new_data:
    bond_df_result = bond_dataframe_filter(bond_prices_df)

# Print bond_df_result

Make sure that you review the documentation for the relevant code and are familiar with how the code above works; in particular, understand the following (and their parameters), as these will all serve you well when you clean, transform, and filter your data in the future (the related documentation is also required reading):

`df.dropna()` <br>
`inplace` <br>
`df.replace()` <br>
`df['X'].astype()` <br>
`pd.to_datetime()` <br>
`df['X'].dt.strftime` <br>
and the code lines that involve the variable `years_mask` <br>
Also, be sure to understand how `df.values.tolist()` works.<br>

## 4. Conclusion

In this lesson, we revisited the calculation for market-implied probability of default but in much more detail. We also downloaded price and rating information for bonds issued by a particular issuer from a well-regarded bond database, FINRA's TRACE.

In the following lesson, we will take the next step toward calculating the market-implied probability of default by estimating the expected cashflows. Then, we can estimate the risk-adjusted discount rate, which equates the bond price to those expected cash flows.



**References**

* Donnelly, Hugh. "Calculating a Company's Probability of Default with Python." AlphaWave Data. https://github.com/AlphaWaveData/Jupyter-Notebooks/blob/master/AlphaWave%20Market-Implied%20Probability%20of%20Default%20Example.ipynb.

* Beygi, Sajjad et al. "Features of a Lifetime PD Model: Evidence from Public, Private, and Rated Firms." Moody's Analytics, May 2018.

* The  code and related documentation used in this lesson is adapted from: <br>**Hugh Donnelly, CFA**<br>*AlphaWave Data* <br> **March 2021** under the following  MIT License:

> Copyright (c) 2020 HDVI <br>Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:<br>The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.<br>THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

**Note:** The above MIT license notice is copied here to comply with its requirements, but it does **not** apply to the content in these lesson notes.





---
Copyright 2023 WorldQuant University. This
content is licensed solely for personal use. Redistribution or
publication of this material is strictly prohibited.
